It is more convenient to run this kernel.\
Running the notebook locally requires downloading all images data.\
https://www.kaggle.com/phananhvu/phan-rsna-pneumonia-pytorch-v0


Transfer Learning for Computer Vision Tutorial
==============================================
**Author**: `Sasank Chilamkurthy <https://chsasank.github.io>`_

In this tutorial, you will learn how to train a convolutional neural network for
image classification using transfer learning. You can read more about the transfer
learning at `cs231n notes <https://cs231n.github.io/transfer-learning/>`__

Quoting these notes,

    In practice, very few people train an entire Convolutional Network
    from scratch (with random initialization), because it is relatively
    rare to have a dataset of sufficient size. Instead, it is common to
    pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
    contains 1.2 million images with 1000 categories), and then use the
    ConvNet either as an initialization or a fixed feature extractor for
    the task of interest.

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initializaion, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.




In [ ]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

import sklearn.model_selection

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import time
import os
import copy
from pathlib import Path
import shutil
import sys

import random
# from PIL import Image, ImageDraw, ImageFont
import PIL
from sklearn.model_selection import train_test_split
import cv2

# plt.ion()   # interactive mode

In [ ]:
print(os.getcwd())
print(os.listdir('../input'))
print(len(os.listdir('../input/rsna-pneumonia-detection-stage-2-jpeg-images/rsna_pneumonia_detection_challenge_images')))
print(os.listdir('../input/lung-opacity-overview'))

In [ ]:
root_dir = Path(os.getcwd())
input_dir = Path('../input')
image_dir = input_dir.joinpath('rsna-pneumonia-detection-stage-2-jpeg-images/rsna_pneumonia_detection_challenge_images')

output_dir = root_dir.joinpath('output')
os.makedirs(output_dir, exist_ok=True)
# os.chdir(root_dir)

print(os.listdir(root_dir))
print(os.listdir(input_dir))
print(os.getcwd())

In [ ]:
print(len(os.listdir(image_dir)))


In [ ]:
# https://www.kaggle.com/kmader/lung-opacity-overview/output?select=image_bbox_full.csv
metadata = pd.read_csv(input_dir.joinpath('lung-opacity-overview/image_bbox_full.csv'))
display(metadata.describe())
display(metadata.sample(5))
print(metadata.shape)

In [ ]:
display(metadata['patientId'].nunique())


In [ ]:
def get_model():
  model = models.mobilenet_v2(pretrained=True)
  num_ftrs = model.classifier[1].in_features
  model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
  # model = models.resnet18(pretrained=True)
  # num_ftrs = model.fc.in_features
  # model.fc=nn.Linear(num_ftrs, len(class_names))
  return model.to(device)


In [ ]:
class_names=['No opacity','Opacity']

In [ ]:
# metadata['path'].apply(lambda path: path[path.rfind('/') + 1:].replace('.dcm', '.jpeg'))

Load Data
---------




In [ ]:
"""
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.Resize((120, 150)),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((120, 150)),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# data_dir = 'data/hymenoptera_data'
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val']}

# use test as train and val as val
image_datasets = {
    'train': datasets.ImageFolder(
        os.path.join(image_dir, 'test'),
        data_transforms['train']),
    'val': datasets.ImageFolder(
        os.path.join(image_dir, 'val'),
        data_transforms['val']),}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
"""

In [ ]:

class ImgDataset(torch.utils.data.Dataset):
    def __init__(self,df,img_dir):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        # self.to_tensor = transforms.ToTensor()
        self.transform = transforms.Compose([
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.Resize(128),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        # First column contains the image paths
        self.df=df
        self.img_dir=img_dir
        self.image_arr = np.asarray(self.df.iloc[:, 0]+'.jpeg')
        # self.image_arr = np.asarray(self.df['path'].apply(lambda path: path[path.rfind('/') + 1:].replace('.dcm', '.jpeg')))
        # Second column is the labels
        self.label_arr = np.asarray(self.df.iloc[:, 5])
        # Third column is for an operation indicator
        #self.operation_arr = np.asarray(self.data_info.iloc[:, 2])
        # Calculate len
        self.data_len = len(self.df.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]

        # Open image
        # img_as_img = PIL.Image.open(self.img_dir+'/'+single_image_name)
        # print(single_image_name)
        img_as_img = PIL.Image.open(self.img_dir.joinpath(single_image_name))

       # print(img_as_img)
        # Check if there is an operation
       # some_operation = self.operation_arr[index]
        # If there is an operation
       # if some_operation:
            # Do some operation on image
            # ...
            # ...
         #   pass
        # Transform image to tensor
        img_transformed = self.transform(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return img_transformed, single_image_label

    def __len__(self):
        return self.data_len


In [ ]:
def filter_data(df):
  filtered_df = df[df['class'] != 'No Lung Opacity / Not Normal']
  filtered_df = filtered_df.drop_duplicates(subset='patientId')
  return filtered_df

filtered_data = filter_data(metadata)
display(filtered_data['class'].value_counts())


In [ ]:
def sample_split(df, nsample=10000, valid_size=0.1):
  samples = df.sample(nsample)
  # display(samples.iloc[:, 5])
  train_df, valid_df = sklearn.model_selection.train_test_split(
      samples, test_size=valid_size, 
      random_state=2021, shuffle=True, 
      stratify=samples['Target'],
  )
  return train_df, valid_df

# train_df, valid_df = sample_split(metadata, nsample=1000, valid_size=0.2)

In [ ]:
# train_ds=ImgDataset(metadata.sample(1000),str(image_dir))
# valid_ds=ImgDataset(metadata.sample(100),str(image_dir))
# train_df, valid_df = sample_split(metadata, nsample=1000, valid_size=0.1)

train_df, valid_df = sklearn.model_selection.train_test_split(
      filtered_data, test_size=0.1, 
      random_state=2021, shuffle=True, 
      stratify=filtered_data['class'],
)

train_ds=ImgDataset(train_df,image_dir)
valid_ds=ImgDataset(valid_df,image_dir)

print(len(train_ds))


In [ ]:
dataloaders = {'val':torch.utils.data.DataLoader(valid_ds, batch_size=64,
                                             shuffle=False, 
                                            #  num_workers=4,
                                             )
              ,
              'train':torch.utils.data.DataLoader(train_ds, batch_size=64,
                                             shuffle=True, 
                                            #  num_workers=4,
                                             )
               }

dataset_sizes = {'val': len(valid_ds),
                 'train':len(train_ds)
                 }
#class_names=metadata['class'].unique() 
# class_names=['No opacity','Opacity']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
inputs, labels = next(iter(dataloaders['train']))
sample_index = np.random.randint(0, len(inputs), 16)

sample_images = inputs[sample_index]
sample_labels = labels[sample_index]
print(sample_images.shape, sample_labels.shape)

In [ ]:
plt.imshow(sample_images[0].numpy().transpose((1, 2, 0)))

In [ ]:
def format_image_for_display(img_tensor):
  return img_tensor.numpy().transpose((1, 2, 0))

Visualize a few images
^^^^^^^^^^^^^^^^^^^^^^
Let's visualize a few training images so as to understand the data
augmentations.



In [ ]:
def show_images(images, labels, class_names=class_names, rows=4, cols=4, figsize=(16, 16),title=None):
    """Imshow for Tensor."""
    # inp = inp.numpy().transpose((1, 2, 0))
    # mean = np.array([0.485, 0.456, 0.406])
    # std = np.array([0.229, 0.224, 0.225])
    # inp = std * inp + mean
    # inp = np.clip(inp, 0, 1)

    # fig, axes = plt.subplots(rows, cols, figsize=(figsize))

    # for nrow, row in enumerate(axes):
    #   for ncol, cell in enumerate(row):
    #       cell.imshow(images[nrow*rows + ncol].numpy().transpose((1, 2, 0)))
    #       cell.set_title(class_names[labels[nrow*rows + ncol]])

    figure = plt.figure(figsize=figsize)
    for idx, img in enumerate(images):
      ax = plt.subplot(rows, cols, idx + 1)
      plt.imshow(img.numpy().transpose((1, 2, 0)))
      plt.title(class_names[labels[idx]])
      plt.axis('off')

    if title is not None:
        plt.title(title)
    # plt.pause(0.001)  # pause a bit so that plots are updated


In [ ]:
show_images(sample_images, sample_labels, figsize=(20, 20))

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    loss_history = []
    score_history = []
    train_loss, train_score, valid_loss, valid_score = [], [], [], []

    for epoch in range(num_epochs):
        print('\n','Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            # end dataloader loop

            if phase == 'train':
                scheduler.step()
#                 loss_history.append(running_loss)
#                 score_history.append(running_corrects)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = (running_corrects.double() / dataset_sizes[phase]).item()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            loss_history, score_history = (train_loss, train_score) if phase == 'train' else (valid_loss, valid_score)
            loss_history.append(epoch_loss)
            score_history.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        # end phase loop

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return dict(   
        model = model,
        train_loss = train_loss,
        train_score = train_score,
        valid_loss = valid_loss,
        valid_score = valid_score,
    )


Visualizing the model predictions
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Generic function to display predictions for a few images




In [ ]:
def visualize_model(model, rows=2, cols=2, figsize=(10, 10)):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure(figsize=figsize)

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            sample_idx = np.random.randint(0, len(inputs), rows*cols)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in (sample_idx):
                images_so_far += 1
                ax = plt.subplot(rows, cols, images_so_far)
                ax.axis('off')
                ax.set_title('prediction: {}\ntruth: {}'.format(class_names[preds[j]], class_names[labels[j]]))
                plt.imshow(inputs.cpu().data[j].numpy().transpose((1, 2, 0)))

                if images_so_far == rows*cols:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [ ]:
 #model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features

# model_ft = models.mobilenet_v2(pretrained=True)
# num_ftrs = model_ft.classifier[1].in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).

In [ ]:
# model_ft.__dict__

In [ ]:

# # model_ft.fc = nn.Linear(num_ftrs, len(class_names))
# #model_ft.classifier[1] = nn.Linear(num_ftrs, len(class_names))
# model_ft=get_model()
# model_ft = model_ft.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Train and evaluate
^^^^^^^^^^^^^^^^^^

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.




In [ ]:
# train_result = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=5)

In [ ]:
# visualize_model(train_result['model'])

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <https://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [ ]:
# model_conv = torchvision.models.resnet18(pretrained=True)
model_conv = models.mobilenet_v2(pretrained=True)
# model_conv = get_model()

for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
num_ftrs = model_conv.classifier[1].in_features

# model_conv.fc = nn.Linear(num_ftrs, 2)
model_conv.classifier[1] = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.classifier[1].parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [ ]:
train_result_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=5)

In [ ]:
# visualize_model(train_result_conv['model'])
# plt.show()

Further Learning
-----------------

If you would like to learn more about the applications of transfer learning,
checkout our `Quantized Transfer Learning for Computer Vision Tutorial <https://pytorch.org/tutorials/intermediate/quantized_transfer_learning_tutorial.html>`_.





# GradCAM
GradCAM, other visualization and utilities \
https://github.com/utkuozbulak/pytorch-cnn-visualizations
\
Part of PyTorch, Less utilities, seems to work better \
https://captum.ai/api/layer.html#gradcam

In [ ]:
model_dir = output_dir/'model'
if not os.path.exists(model_dir):
  os.makedirs(model_dir)

# model_path = model_dir/'mobilenetv2-imgsize128.pth'
model_path = model_dir/'mobilenetv2-imgsize128.pth'

In [ ]:
torch.save(train_result_conv['model'].state_dict(), model_path)


In [ ]:
# class_names=["Normal","Pneumonia"]

In [ ]:
# trained_model = get_model()
# trained_model.load_state_dict(torch.load(model_path))
# # trained_model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

# # trained_model=train_result['model']

trained_model = train_result_conv['model'].eval()

In [ ]:
def predict(model, inputs):
  outputs = model(inputs)
  _, preds = torch.max(outputs, 1)
  return preds


In [ ]:
sample_predictions = predict(trained_model, sample_images.to(device))
print(sample_predictions.shape)

In [ ]:
# activation = {}
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook

# layer = list(trained_model.features.children())[-1]
# hook1 = layer.register_forward_hook(get_activation('lastconv'))

# lastconv_acts = []
# # go through all the batches in the dataset
# for X, y in dataloaders['val']:
#   # forward pass -- getting the outputs
#   outpt = trained_model(X.to(device))

#   # collect the activations in the correct list
#   # https://github.com/pytorch/vision/blob/3ee34eb1559f049dfe96581353bef29af17d95f3/torchvision/models/mobilenetv2.py#L192
#   avg_acts = ( nn.functional.adaptive_avg_pool2d(activation['lastconv'], (1, 1))
#     .reshape(activation['lastconv'].shape[0], -1) )
#   lastconv_acts.append(avg_acts)

#   del outpt
#   # del avg_acts
#   # gc.collect()
                      
# lastconv_acts = torch.cat(lastconv_acts, dim=0)
# # lastconv_acts = np.concatenate((lastconv_acts))
# hook1.remove()

In [ ]:
# print(lastconv_acts.shape)
# print(lastconv_acts[0].shape)

In [ ]:
# conv_features = pd.concat((valid_df['patientId'].reset_index(drop=True), pd.DataFrame(lastconv_acts.cpu().numpy())), axis=1)
# print(conv_features.shape)
# display(conv_features.sample(5))


In [ ]:
# feature_dir = output_dir/'feature'
# os.makedirs(feature_dir, exist_ok=True)

# conv_feature_path = feature_dir/'sample-conv-feature.csv'

In [ ]:
# conv_features.to_csv(conv_feature_path)

In [ ]:
!pip install captum

In [ ]:
import captum.attr
import scipy.ndimage.interpolation


In [ ]:
trained_model = train_result_conv['model']
layer_gradcam = captum.attr.LayerGradCam(trained_model, list(trained_model.features.children())[-1])


In [ ]:
gradcam_attr = layer_gradcam.attribute(sample_images[0].unsqueeze(0).to(device), sample_labels[0])
print(gradcam_attr, gradcam_attr.shape)


In [ ]:
print(np.array(sample_images[0].shape)/np.array(gradcam_attr.shape[1:]))
print(gradcam_attr.cpu().detach().numpy().shape)

In [ ]:
# upsample CAM to original image size
gradcam_upsample = scipy.ndimage.interpolation.zoom(gradcam_attr.squeeze(0).cpu().detach().numpy(), np.array(sample_images[0].shape)/np.array(gradcam_attr.shape[1:]))

In [ ]:
print(gradcam_upsample[0,30:35, 75:80])
print(gradcam_upsample[0,-5:, -5:])

In [ ]:

_ = captum.attr.visualization.visualize_image_attr_multiple(
  # np.transpose(gradcam_attr.squeeze(0).cpu().detach().numpy(), (1,2,0)),
  np.transpose(gradcam_upsample, (1,2,0)), 
  format_image_for_display(sample_images[0]),
  methods = 'original_image blended_heat_map'.split(' '),
  signs='all all'.split(' '),
  alpha_overlay=0.6,
  cmap='viridis',
)


In [ ]:
layer_gradcam = captum.attr.LayerGradCam(trained_model, list(trained_model.features.children())[-1])
def plot_gradcam(layer_gradcam=layer_gradcam, index=0):
  gradcam_attr = layer_gradcam.attribute(sample_images[index].unsqueeze(0).to(device), sample_labels[index])
  
  # upsample CAM to original image size
  gradcam_upsample = scipy.ndimage.interpolation.zoom(
      gradcam_attr.squeeze(0).cpu().detach().numpy(), 
      np.array(sample_images[0].shape)/np.array(gradcam_attr.shape[1:]),
  )
  
  _ = captum.attr.visualization.visualize_image_attr_multiple(
    np.transpose(gradcam_upsample, (1,2,0)), 
    format_image_for_display(sample_images[index]),
    methods = 'original_image blended_heat_map'.split(' '),
    signs='all all'.split(' '),
    alpha_overlay=0.6,
    cmap='viridis',
    titles=[f'truth: {class_names[sample_labels[index]]}', f'prediction: {class_names[sample_predictions[index]]}'],
    show_colorbar=True
  )


In [ ]:
plot_gradcam(index=1)

In [ ]:
train_metric = {key: value 
                for key, value in train_result_conv.items()
                if 'loss' in key or 'score' in key}
print(train_metric)

In [ ]:
import pickle


In [ ]:
with open(model_dir/'train_metric.pkl', 'wb') as file:
    pickle.dump(train_metric, file)

In [ ]:
print(os.listdir(output_dir))
print(os.listdir(model_dir))
# print(os.listdir(feature_dir))

In [ ]:
shutil.make_archive('output', 'zip', model_dir)
# shutil.rmtree(model_dir, ignore_errors=True)
print(os.listdir(root_dir))
